# GPX Viewer

In [ ]:
import json
from statistics import mean

import gpxpy
import srtm

from ipyleaflet import Map, Polyline, Marker
from ipyupload import FileUpload 
from ipywidgets import HTML, Output, IntSlider

In [ ]:
def parse_data(file):
    gpx = gpxpy.parse(file)
    elevation_data = srtm.get_data()
    elevation_data.add_elevations(gpx, smooth=True)
    return gpx

In [ ]:
out = Output()

In [ ]:
@out.capture()
def plot_map(gpx):
    points = [p.point for p in gpx.get_points_data()]
    mean_lat = mean(p.latitude for p in points)
    mean_lng = mean(p.longitude for p in points)

    # create map
    m = Map(center=(mean_lat, mean_lng), zoom=15)

    # show trace
    line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                    color = "red", fill=False)
    m.add_layer(line)

    # add markers
    for point in gpx.waypoints:
        lat, lng = point.latitude, point.longitude
        marker = Marker(location=(lat, lng), title=point.name, popup=HTML(value=point.name), draggable=False)
        m.add_layer(marker);

    display(m)

In [ ]:
@out.capture()
def parse_files(change):
    # unpack data
    all_files = [v for _, v in change['new'].items()]
    from io import StringIO
    raw = all_files[0]['content'].decode('utf-8')
    with StringIO(raw) as f:
        gpx = parse_data(f)
    
    print(gpx.name)
    plot_map(gpx)

In [ ]:
uploader = FileUpload(accept=None,
               multiple=False, 
               disabled=False,
               style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
               compress_level=9
              )

uploader.observe(parse_files, names='value')

In [ ]:
with out:
    display(uploader)
out